https://blog.fastforwardlabs.com/2018/04/10/pytorch-for-recommenders-101.html

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import utils

import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
device

In [ ]:
rating,_,_ = utils.get_movielens_data()
n_users = rating["userid"].max() + 1
n_items = rating["itemid"].max() + 1
print(n_users, n_items)
train_df, test_df = utils.movielens_train_test_split(rating)
train_df.shape, test_df.shape

# Model

In [ ]:
n_users

In [ ]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)

    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [ ]:
model = MatrixFactorization(n_users, n_items, n_factors=20)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

model = model.to(device)

In [ ]:
trainloader = DataLoader(train_df.values, batch_size=128)
testloader = DataLoader(test_df.values, batch_size=128)

In [ ]:
def test_method():
    maes = []
    for batch in tqdm(testloader):
        user = batch[:, 0].type(torch.LongTensor).to(device)
        item = batch[:, 1].type(torch.LongTensor).to(device)

        score = batch[:, 2].type(torch.FloatTensor)
        prediction = model(user, item)
        mae = np.mean(np.abs((prediction.cpu() - score).detach().numpy()))
        maes.append(mae)
    return np.mean(maes)

In [ ]:
losses = []
for epoch in range(50):
    losses_epoch = []
    for batch in tqdm(trainloader):
        user = batch[:, 0].type(torch.LongTensor).to(device)
        item = batch[:, 1].type(torch.LongTensor).to(device)

        score = batch[:, 2].view(-1, 1).type(torch.FloatTensor)
        prediction = model(user, item)
        loss = loss_fn(prediction, score.to(device))
        loss.backward()
        optimizer.step()
        losses_epoch.append(loss.item())
    losses.append(np.mean(losses_epoch))
    print(losses[-1])
    print("TEST ", test_method())